# Classifying images

This tutorial is based on the excellent TensorFlow tutorials ([MNIST for ML Beginners](https://www.tensorflow.org/get_started/mnist/beginners), [Deep MNIST for Experts](https://www.tensorflow.org/get_started/mnist/pros)), as well as the corresponding Keras examples ([MNIST MLP](https://github.com/fchollet/keras/blob/master/examples/mnist_mlp.py), [MNIST CNN](https://github.com/fchollet/keras/blob/master/examples/mnist_cnn.py)).

MNIST is a dataset of images of digits from 0-9, gathered from human handwriting. Our goal is to train a computer to "read" these digits. Considered the "Hello World" of machine learning, **classifying MNIST** is a good example task that will teach us the general framework of machine learning using deep learning frameworks.

## Single layer neural network using TensorFlow

This section corresponds to [MNIST for ML Beginners](https://www.tensorflow.org/get_started/mnist/beginners). The goal is to write a Softmax Regression model using TensorFlow.

In [1]:
import tensorflow as tf

In [2]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [3]:
x = tf.placeholder(tf.float32, [None, 784])
W = tf.Variable(tf.zeros([784, 10]))
b = tf.Variable(tf.zeros([10]))
y = tf.nn.softmax(tf.matmul(x, W) + b)

In [4]:
y_ = tf.placeholder(tf.float32, [None, 10])

loss = tf.reduce_mean(
      tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.5)
train = optimizer.minimize(loss)


In [5]:
sess = tf.InteractiveSession()
tf.global_variables_initializer().run()
for _ in range(1000):
  batch_xs, batch_ys = mnist.train.next_batch(100)
  sess.run(train, feed_dict={x: batch_xs, y_: batch_ys})

In [6]:
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print(sess.run(accuracy, feed_dict={x: mnist.test.images, y_: mnist.test.labels}))


0.9065


## Single layer neural network using Keras

Now that we understand the lower level, let's see how we can quickly do the same thing using the higher-level library Keras.

In [7]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Conv2D, MaxPooling2D, Flatten
from keras.optimizers import SGD, Adam

Using TensorFlow backend.


In [8]:
model = Sequential()
model.add(Dense(10, activation='softmax',input_shape=(784,)))
model.compile(loss='categorical_crossentropy',
              optimizer=SGD(lr=0.5),
              metrics=['accuracy'])

In [9]:
batch_size = 100
epochs = 1

model.fit(mnist.train.images, mnist.train.labels, epochs=epochs,batch_size=batch_size,
                    verbose=1, validation_data=(mnist.test.images, mnist.test.labels))

score = model.evaluate(mnist.test.images, mnist.test.labels, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 55000 samples, validate on 10000 samples
Epoch 1/1
55000/55000 [==============================] - 1s - loss: 0.4011 - acc: 0.8857 - val_loss: 0.3009 - val_acc: 0.9151
Test loss: 0.300912593853
Test accuracy: 0.9151


## Convolutional neural network using Keras

Now, let's use Keras to build a better model, the convolutional neural network.

In [10]:
# CNN with Keras

x_train = mnist.train.images.reshape(mnist.train.images.shape[0], 28, 28, 1)
x_test = mnist.test.images.reshape(mnist.test.images.shape[0], 28, 28, 1)

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=[28,28,1]))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

In [ ]:
batch_size = 128
epochs = 1

model.fit(x_train, mnist.train.labels, epochs=epochs,batch_size=batch_size,
                    verbose=1, validation_data=(x_test, mnist.test.labels))

score = model.evaluate(x_test, mnist.test.labels, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [ ]:
model.save('mnist_cnn.h5')
del model

In [12]:
model = keras.models.load_model('mnist_cnn.h5')
final_loss, final_accuracy = model.evaluate(x_test, mnist.test.labels)
print('Accuracy', final_accuracy)

 9920/10000 [============================>.] - ETA: 0sAccuracy 0.9812
